# Limpeza de Dados para Análise Exploratória

Este notebook realiza uma limpeza básica no dataset de treino. O objetivo é preparar os dados para **análise exploratória (EDA)**, e não para treinar um modelo de machine learning. 

As seguintes etapas são realizadas:
1. Remoção de linhas duplicadas.
2. Tratamento de valores ausentes (NaN).
3. Remoção de outliers em colunas numéricas.
4. Agrupamento de categorias raras em colunas categóricas.

O dataset resultante será salvo em `data/train_data_analysis_v2.csv`.

## 1. Importação das Bibliotecas

Primeiro, importamos as bibliotecas necessárias. `pandas` é essencial para manipulação de dados e `numpy` para operações numéricas.

In [ ]:
import pandas as pd
import numpy as np

## 2. Carregamento dos Dados

Carregamos o dataset original a partir do arquivo `train_data.csv`.

In [ ]:
df = pd.read_csv('../data/train_data.csv')

## 3. Limpeza dos Dados

Agora, aplicamos as técnicas de limpeza.

### 3.1. Remoção de Duplicatas

Linhas duplicadas podem distorcer a análise, então garantimos que cada registro seja único.

In [ ]:
df_limpo = df.drop_duplicates()
print(f'Linhas removidas por duplicidade: {len(df) - len(df_limpo)}')

### 3.2. Tratamento de Valores Ausentes

Verificamos se há valores ausentes (NaN) e os tratamos. Para a análise, preenchemos os valores ausentes de `Bed Grade` com a moda (o valor mais frequente) e os de `City_Code_Patient` com 0, assumindo que `0` pode representar 'Não informado'.

In [ ]:
df_limpo['Bed Grade'] = df_limpo['Bed Grade'].fillna(df_limpo['Bed Grade'].mode()[0])
df_limpo['City_Code_Patient'] = df_limpo['City_Code_Patient'].fillna(0)

### 3.3. Remoção de Outliers (Método IQR)

Outliers são valores extremos que podem distorcer as métricas estatísticas. Usamos o método do Intervalo Interquartil (IQR) para identificar e remover outliers das colunas numéricas.

In [ ]:
num_cols = df_limpo.select_dtypes(include=np.number).columns
linhas_antes = len(df_limpo)

for col in num_cols:
    Q1 = df_limpo[col].quantile(0.25)
    Q3 = df_limpo[col].quantile(0.75)
    IQR = Q3 - Q1
    lim_inf = Q1 - 1.5 * IQR
    lim_sup = Q3 + 1.5 * IQR
    
    df_limpo = df_limpo[(df_limpo[col] >= lim_inf) & (df_limpo[col] <= lim_sup)]

print(f'Linhas removidas por outliers: {linhas_antes - len(df_limpo)}')

### 3.4. Agrupamento de Categorias Raras

Categorias com pouquíssimas ocorrências podem não ser estatisticamente significantes para análise. Agrupamos todas as categorias que representam menos de 1% do total em uma nova categoria chamada 'OUTRA'.

In [ ]:
cat_cols = df_limpo.select_dtypes(include=['object']).columns

for col in cat_cols:
    freq = df_limpo[col].value_counts(normalize=True)
    raras = freq[freq < 0.01].index
    
    if len(raras) > 0:
        df_limpo[col] = df_limpo[col].replace(raras, 'OUTRA')
        print(f'Categorias raras agrupadas na coluna: {col}')

## 4. Salvando o Dataset Limpo

Finalmente, salvamos o dataframe limpo em um novo arquivo CSV para ser usado na análise exploratória.

In [ ]:
df_limpo.to_csv('../data/train_data_analysis_v2.csv', index=False)
print('\nDataset salvo com sucesso em ../data/train_data_analysis_v2.csv')